<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/feature-pm/ML_structure_project_ScaledData_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Load the data



In [35]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os 

#first put a shortcut in your drive to the image processing folder

#Choose the image folder you want to try

pipeFolder = '10.8Prepro+haar+fixed+CorrectedLabels_full'

RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        pipeFolder)

RESULTS_DIR_MAIN = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results')

MODELS_DIR =  os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        'MLModelsFinal')



DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')


print(os.listdir(RESULTS_DIR))

results_file = os.listdir(RESULTS_DIR)

['20586934_6c613a14b80a8591_MG_L_CC_ANON__haar_fos_glcm.csv', '20586960_6c613a14b80a8591_MG_R_ML_ANON__haar_fos_glcm.csv', '20586986_6c613a14b80a8591_MG_L_ML_ANON__haar_fos_glcm.csv', '20587054_b6a4f750c6df4f90_MG_R_CC_ANON__haar_fos_glcm.csv', '20587080_b6a4f750c6df4f90_MG_R_ML_ANON__haar_fos_glcm.csv', '20587148_fd746d25eb40b3dc_MG_R_CC_ANON__haar_fos_glcm.csv', '20587174_fd746d25eb40b3dc_MG_L_CC_ANON__haar_fos_glcm.csv', '20587200_fd746d25eb40b3dc_MG_R_ML_ANON__haar_fos_glcm.csv', '20587226_fd746d25eb40b3dc_MG_L_ML_ANON__haar_fos_glcm.csv', '20587294_e634830794f5c1bd_MG_R_CC_ANON__haar_fos_glcm.csv', '20587320_e634830794f5c1bd_MG_L_CC_ANON__haar_fos_glcm.csv', '20587346_e634830794f5c1bd_MG_R_ML_ANON__haar_fos_glcm.csv', '20587372_e634830794f5c1bd_MG_L_ML_ANON__haar_fos_glcm.csv', '20587466_d571b5880ad2a016_MG_L_CC_ANON__haar_fos_glcm.csv', '20587492_d571b5880ad2a016_MG_R_ML_ANON__haar_fos_glcm.csv', '20587518_d571b5880ad2a016_MG_L_ML_ANON__haar_fos_glcm.csv', '20587544_d571b5880ad2a

In [37]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

In [38]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [39]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import tree

import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
import pickle
from sklearn.calibration import CalibratedClassifierCV


In [40]:
from sklearn.ensemble import RandomForestClassifier # random forest
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import copy

In [41]:
import pandas as pd
import os
import cv2
#!pip install fastprogress
from fastprogress import master_bar, progress_bar
#!pip install tqdm
from tqdm.notebook import tqdm_notebook
from skimage import measure
from matplotlib import pyplot as plt

In [42]:
def read_results_train_test(path, keys):
  df_result = pd.DataFrame()
  for result in os.listdir(path):
    if result.split('.')[-1] == 'csv':
      if int(result.split('_')[0]) in list(keys):
        try:
          df = pd.read_csv(path+'/'+result)
          df_result = df_result.append(df)
        except:
          print("Empty file ", result)
  return df_result

In [43]:
def read_results(path):
  dfs = pd.DataFrame()
  for result in results_file:
    try:
      df = pd.read_csv(path+'/'+result)
    except:
      print("Empty file ", result)

    dfs = dfs.append(df)
  
  return dfs


In [44]:
def train_test_function():
  df_train_test = pd.read_csv(os.path.join('/content',
                                'drive',
                                'MyDrive',
                                'Results',
                                'standard_partitions.csv'), index_col=0)
  train_keys = df_train_test.loc[df_train_test.partition == 'train']['image_id'].values
  test_keys = df_train_test.loc[df_train_test.partition == 'test']['image_id'].values


  dfs_train = read_results_train_test(RESULTS_DIR, train_keys)
  dfs_test = read_results_train_test(RESULTS_DIR, test_keys)
  return dfs_train,dfs_test

In [45]:
# dfs_train, dfs_test = train_test_function()
# dfs_train.label.value_counts()

# **Preprocessing**

In [46]:
# We will balance data randomly taking  the same amount of 0 and 1 label inputs. There are several options to manage unbalanced data,
# this is just one approach, we should try different approaches.
# https://towardsdatascience.com/comparing-different-classification-machine-learning-models-for-an-imbalanced-dataset-fdae1af3677f
def rus(dfs, not_consider=0):
  # df_negative = dfs.loc[dfs.label == dfs.label.value_counts().idxmax()] #This is for making it more general
  df_negative = dfs.loc[dfs.label == 0] #Take all negative samples from dfs -> label = 0
  print('rus negative 1: ', len(df_negative))
  print('rus negative 1 without duplicates: ', len(df_negative.drop_duplicates()))
  #Here we are taking a subset from the 'negative pool' called df_negative with length equal to the number of positive candidates I have
  if not_consider >= len(dfs.loc[dfs.label == 1]):
    df_0 = pd.DataFrame()
  else:
    df_0 = df_negative.sample(len(dfs.loc[dfs.label == 1])-not_consider, random_state = 1) #not consider is the number of negative samples that I already have and that I misclassified the first time
  print('df_0: ', len(df_0))
  print('append ', len(df_negative.append(df_0)))
  print('df_negative drop no keeping', len(df_negative.drop_duplicates(keep=False)))
  df_negative = df_negative.append(df_0).drop_duplicates(keep=False) #df_negative is the samples that have not been used, and have not been selected yet for the training
  print('after drop duplicates with appended: ',len(df_negative))
  dfs_eq = df_0.append(dfs.loc[dfs.label == 1]) #dfs.label.value_counts().idxmin() #Here we are taking the balanced tada
  print("Length of balanced data: ", len(dfs_eq))
  print(dfs_eq['label'].value_counts())
  return dfs_eq, df_negative

In [47]:
def plotFeatures(data):
  #plot the scatter matrix
  pd.plotting.scatter_matrix(data,figsize=(25,25))
  #correlation plot
  corr = data.corr()
  f, ax = plt.subplots(figsize=(25, 25))
  sns.heatmap(corr,annot=True)

In [48]:
def standardScaler(X_train, X_test):
  standard_scaler = StandardScaler()
  X_train = standard_scaler.fit_transform(X_train)
  X_test = standard_scaler.transform(X_test)
  return X_train, X_test

# **Feature Engineering**

In [49]:
def featureSelectionTrees(estimators, X_train, y_train, X_test):
  clf = ExtraTreesClassifier(n_estimators=estimators)
  clf = clf.fit(X_train, y_train)
  model = SelectFromModel(clf, prefit=True)
  X_train = model.transform(X_train)
  X_test = model.transform(X_test)
  return X_train, X_test

In [50]:
def pcaAnalysis(components, X_train, X_test):
  pca = PCA(components)
  X_train = pca.fit_transform(X_train)
  X_test = pca.transform(X_test)
  print("PCA variance ratio: ", pca.explained_variance_ratio_)
  print("Total variance Explained by PCA: ", sum(pca.explained_variance_ratio_))
  return X_train, X_test

In [51]:
def selectFeaturesChi(k, X_train, X_test, y_train):
  sel = SelectKBest(chi2, k=k)
  sel.fit(X_train, y_train)
  X_train = sel.transform(X_train)
  X_test = sel.transform(X_test)
  return X_train, X_test

# **Classification**

In [52]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.base import clone
from sklearn.metrics import matthews_corrcoef

In [53]:

def RandomForest(X_train, y_train, cv=5, best_params = dict()):
  if len(best_params) == 0:
    print("Searching for best hyperparameters")
    params = {'criterion': ['gini'],
              'n_estimators': [100], # , 500, 900
              'max_features': ['auto', 'sqrt'],#, 'sqrt', 'log2'
              'max_depth' : [10, 12]}
    grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs = -1), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

    print("The best parameters for rf are %s with an accuracy of %0.4f"%(best_params, grid.best_score_))
  
  classifier = RandomForestClassifier(random_state=42, n_jobs = -1)
  
  return classifier, best_params

In [54]:
from sklearn.ensemble import GradientBoostingClassifier # gradient boosting regressor
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
def GradientBoosting(X_train, y_train, cv=5, best_params = dict()):
  if len(best_params) == 0:
    params = {'learning_rate': [0.1],#0.05, 0.2
              #'min_samples_split': [0.5, 0.8],
              #'min_samples_leaf': [0.1, 0.2, 0.5],
              'max_depth':[8],
              #'max_features':['sqrt'],#'log2'
              #'criterion': ['friedman_mse',  'mae'],
              #'subsample':[0.5, 1.0],
              'n_estimators':[600]}
    grid = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

    print("The best parameters for gb are %s with an accuracy of %0.4f"%(best_params, grid.best_score_))

  classifier = GradientBoostingClassifier(random_state=42)
  
  return classifier, best_params

In [55]:
from sklearn.ensemble import AdaBoostClassifier # adaboost, adaboostRegressor for regression problems

def AdaBoost(X_train, y_train, cv=5, best_params = dict()):
  if len(best_params) == 0:
    params = {'n_estimators': [600],
              'learning_rate': [0.5]}
    grid = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

    print("The best parameters for ab are %s with an accuracy of %0.4f"%(best_params, grid.best_score_))
  
  classifier = AdaBoostClassifier(random_state=42)
  return classifier, best_params

In [56]:
from sklearn.linear_model import LogisticRegression

def LogRegre(X_train, y_train, cv=5, best_params = dict()):
  if len(best_params) == 0:
    params = {'multi_class':['multinomial'],
              'solver': ['sag'],#,'saga'
              'penalty': ['l2']}#'elasticnet',
    grid = GridSearchCV(LogisticRegression(random_state=42), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

    print("The best parameters for lrg are %s with an accuracy of %0.4f"%(best_params, grid.best_score_))
  
  classifier = LogisticRegression(random_state=42)
  
  return classifier, best_params

In [57]:
def KNN(X_train, y_train, cv=5, best_params = dict()):
  if len(best_params) == 0:
    params = {'n_neighbors':[3,5]}
    grid = GridSearchCV(KNeighborsClassifier(), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

  classifier = KNeighborsClassifier()
  return classifier, best_params

In [58]:
def DecisionTree(X_train, y_train, cv=5, best_params = dict()):
  classifier = tree.DecisionTreeClassifier()
  #classifier = CalibratedClassifierCV(base_estimator=dtclf, cv='prefit')
  return classifier, best_params

In [59]:
# This is an example of how to use a Pipe inside a function we are training, as done in Challenge 3 by the professor
def SVC_linear(X_train, y_train, cv=5, best_params = dict()):

  if len(best_params) == 0:
    lower_value_C = 1
    higher_value_C = 10
    n_values = 10
    base = 10
    params = {'C': [0.1, 1, 3, 5,9,11], #12
              'kernel' : ['rbf'],
              'gamma': [2.5, 5, 10]}

    grid = GridSearchCV(SVC(random_state = 42, probability=True), param_grid=params, cv=cv, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_

    print("The best parameters for svm are %s with an accuracy of %0.4f"%(best_params, grid.best_score_))
  
  classifier = SVC(random_state = 42, probability=True)
  #classifier = CalibratedClassifierCV(base_estimator=svcclf, cv='prefit')
  return classifier, best_params

In [60]:
def negative_pool(dataBalanced, dfNegative, hyperOptBalanceData, X_test, y_test, classifier, calibrated=False, hyperParam=dict() ):
  count = 0
  f1_scores = []
  pAUCs = []

  trainBalanceData = dataBalanced.drop('label', axis=1)
  trainBalanceLabel = dataBalanced['label']

  hyperOptTrainData = hyperOptBalanceData.drop('label', axis=1)
  hyperOptLabel = hyperOptBalanceData['label']


# Test on the training set to check for overfitting
# We can try with a cascade - Viola Jones
# Adaboost for classifier - Rankboost
# Nor PCA because not a lot of features
# Correlation check.

  while True:

    #Classification

    if count==0:

      print(f"First iteration {classifier}... looking for best hyperparameters")

      if classifier == 'svm':
        clf, best_params = SVC_linear(hyperOptTrainData, hyperOptLabel, best_params = hyperParam)
      elif classifier == 'rf':
        clf, best_params = RandomForest(hyperOptTrainData, hyperOptLabel, best_params = {'criterion': 'gini', 'max_depth': 12, 'max_features': 'auto', 'n_estimators': 100})
      elif classifier == 'gb':
        clf, best_params = GradientBoosting(hyperOptTrainData, hyperOptLabel, best_params = {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 600})
      elif classifier == 'adaboost':
        clf, best_params = AdaBoost(hyperOptTrainData, hyperOptLabel, best_params = {'learning_rate': 0.5, 'n_estimators': 600})
      elif classifier == 'logreg':
        clf, best_params = LogRegre(hyperOptTrainData, hyperOptLabel, best_params = hyperParam)
      elif classifier == 'knn':
        clf, best_params = KNN(hyperOptTrainData, hyperOptLabel, best_params = hyperParam)
      elif classifier == 'dt':
        clf, best_params = DecisionTree(hyperOptTrainData, hyperOptLabel,best_params = hyperParam)

      clf.set_params(**best_params)    
      if calibrated:
        clf.fit(trainBalanceData, trainBalanceLabel)
        clf = CalibratedClassifierCV(base_estimator=clf, cv='prefit')
    

    clf.fit(trainBalanceData, trainBalanceLabel)
    pred = clf.predict(trainBalanceData)


    print("TRAIN")
    print('F1-score train: ', f1_score(trainBalanceLabel, pred))
    y_pred_proba_tr = clf.predict_proba(trainBalanceData)[:, 1]
    print('pAUC train: ', roc_auc_score(trainBalanceLabel, y_pred_proba_tr, max_fpr = 0.0001))
    print(confusion_matrix(trainBalanceLabel, pred))

    print("TEST")
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)[:, 1]

    print('F1-score test: ', f1_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print('pAUC test: ', roc_auc_score(y_test, y_pred_proba, max_fpr = 0.0001))
    print(confusion_matrix(y_test, y_pred))

    pAUCs.append(roc_auc_score(y_test, y_pred_proba, max_fpr = 0.0001))

    misclassified = pd.DataFrame(columns = trainBalanceData.columns)
    labels_pool = []
    print(len(pred))
    print(len(trainBalanceLabel))
    for i in range(len(pred)):
      if (pred[i]==1) & (trainBalanceLabel.iloc[i] == 0): #FP False positive
#        new_row = pd.DataFrame(X_test[i].reshape(1, data.shape[1] ), columns=data.columns)
        new_row = trainBalanceData.iloc[i]
        misclassified = misclassified.append(new_row)
        labels_pool.append(trainBalanceLabel.iloc[i])

    dfPositive = dataBalanced.loc[dataBalanced.label == 1]

    if len(dfPositive)*0.9 >= (len(dfNegative)+len(misclassified)):
      print("Negative pool is too short...")
      print("Len of positive samples: ", len(dfPositive))
      print("Len of negative samples: ", len(dfNegative)+len(misclassified))
      break

    if (len(f1_scores) > 5 and (f1_scores[-1] - f1_scores[-2])< 0.001): 
      print('last ', f1_scores[-1])
      print('second last ', f1_scores[-2])
      print("No significant change in 5 iterations. Escaping")
      break

    dataBalanced, dfNegative = rus(dfPositive.append(dfNegative), len(misclassified)) #Get a new pool of dfNegative
    
    trainBalanceLabel = dataBalanced['label'].append(pd.Series(labels_pool))
    trainBalanceData = dataBalanced.drop(['label'], axis=1).append(misclassified)

    print('Data len: ', len(trainBalanceData))
    print('labels len: ', len(trainBalanceLabel))
    
    count +=1

  return clf, best_params, y_pred_proba, y_pred, np.mean(f1_scores)

# **Scores**

In [61]:
import os 

#first put a shortcut in your drive to the image processing folder

DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')


print(os.listdir(DATA_DIR))

['normals.txt', 'images', 'groundtruths', 'masks']


In [62]:
_, _, groundTruths = next(os.walk(os.path.join(DATA_DIR, 'groundtruths')))

In [63]:
from numpy.lib.function_base import average
import cv2

def calculateFROC(groundTruthsList, normals, candidates, prediction, model, pipeline):

  #List needed for counting
  evaluationList = []

  # evaluation froc curve #

  fn = 0 # false negative, for the blobs that do not belong to any component
  positive_candidates = 0
  flag = True

  rowListdfROC = []

  candidates_copy= candidates.copy()
  candidates_copy['prediction'] = prediction

  for imageKey in tqdm_notebook(groundTruthsList):


    evaluationList = []
 
    # list of features found with y,x and sigma
    candidatesImg = candidates_copy.loc[candidates_copy['name'].str.contains(imageKey.split('_')[0], regex=False)]
    candidates_number = len(candidatesImg)
#    print('cand number: ', candidates_number)

    mask = cv2.imread(os.path.join(DATA_DIR, 'groundtruths', imageKey), cv2.IMREAD_GRAYSCALE)

 #   print('# IMAGEKEY: ', imageKey)
    blobs = mask > 0.7 * mask.mean() #Thresholding the backgroudnd
    blobs_labels, count = measure.label(blobs, background=0, return_num=True) #Getting labels of the connected components and the amount of them without considering the count

    # dictionaries
    dictCounting={}

  #  print(count)
    for index in range(1, count+1):
      dictCounting[index] = 0
    
    evaluationList = []
    dictMean = {}

    for index2, candidate in candidatesImg.iterrows():
      sigma = 7
      
      if (imageKey.split('_')[0] in normals):
        if candidate['name'].split('_')[1] not in dictMean.keys():
          dictMean[candidate['name'].split('_')[1]] = [candidate.prediction]
        else:
          dictMean[candidate['name'].split('_')[1]].append(candidate.prediction)
        continue
      

      # is the image a normal image?

      if (((candidate.x - sigma) < 0) or 
        ((candidate.x + sigma) > mask.shape[0]) or 
        ((candidate.y - sigma) < 0) or
        ((candidate.y + sigma) > mask.shape[1])):
        continue


      # n = 3
      left = int(candidate.x - sigma)
      right = int(candidate.x + sigma)
      top = int(candidate.y - sigma)
      bottom = int(candidate.y + sigma) 

      #    y : y + w , x : x +  h
      nonzero = cv2.countNonZero(blobs_labels[top:bottom, left: right])

      if nonzero > 0:
        # Find all connected components (cc) that intersect with the candidate
        foundCC = [i for i in np.unique(blobs_labels[top:bottom, left: right]) if i!= 0]

        # Keep the maximum prediction of the candidates that intsersect with at least one component
        for cc in foundCC:
          dictCounting[cc] = max(candidate.prediction, dictCounting[cc])

    if (imageKey.split('_')[0] not in normals):
      for key,value in dictCounting.items():
        if np.sum(blobs_labels == key) > np.floor(np.pi*(15/2.0)**2):
#          print("Too big... discarded")
          continue
        if value > 0:
            rowListdfROC.append(['TP', value])
        else:
          fn = fn + 1
    else:
      if len(dictMean.keys()) > 0:
        for key, value in dictMean.items():
          rowListdfROC.append(['FP', np.max(np.array(value))])


  filename_key = 'FROC_calculations_{}_pip{}_fn{}_normals{}_negative_pool_scaled.csv'.format(model, pipeline, fn, len(normals))

  dfROC = pd.DataFrame(rowListdfROC, columns=['type', 'prob'])

  dfROC.to_csv(os.path.join(MODELS_DIR,
                            'negative_pool_scaled',
                            filename_key))
  
  print("File saved as ", filename_key)



  return fn, dfROC, filename_key

In [64]:
def writeFile(df, flag, name):
  if(flag):
    df.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 name),
                    mode='a',
                    index=False)
    flag = False
  else:
    df.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 name),
                  mode='a',
                  header=False,
                  index=False)
  return flag

In [65]:
import matplotlib.pyplot as plt
from numpy import trapz

def draw_curve(fn, normals, dfROC, name, clf_name):

  experiments_results_path = os.path.join(MODELS_DIR, 'negative_pool_scaled')
  results_filename = f'results_AUC_negative_pool_scaled_{pipeFolder}.csv'
  confusion_matrix_filename = f'results_ConfMat_negative_pool_scaled_{pipeFolder}.csv'

  if results_filename not in os.listdir(experiments_results_path):
    results_df = pd.DataFrame(columns=["model", "AUC_TOTAL", "AUC_final_50fPpi", "F1_final"])
    results_df.to_csv(os.path.join(experiments_results_path, results_filename), index=False)
    
    results_confmat = pd.DataFrame(columns=["model", "TP", "FP", "FN", "Total True"])
    results_confmat.to_csv(os.path.join(experiments_results_path, confusion_matrix_filename), index=False)

  else:
    results_df = pd.read_csv(os.path.join(experiments_results_path, results_filename))
    results_confmat = pd.read_csv(os.path.join(experiments_results_path, confusion_matrix_filename))

  tpc = 0
  fpc = 0
  tpr = []
  fppi = []

  dfROC['prob'] = [round(i,4) for i in dfROC['prob'].values]
  dfROC = dfROC.sort_values('prob', ascending=False)

  thresholds = dfROC.prob.unique()
  print('Number of thresholds: ', len(thresholds))
  print(thresholds)


  tp = len(dfROC.loc[dfROC.type == 'TP'])
  print('true positives: ', tp)
  fp = len(dfROC.loc[dfROC.type == 'FP'])
  print('false positives: ',  fp)
  print("Total number of positives: ", tp+fn)

  for i in progress_bar(range(len(thresholds))):

    tpc += len(dfROC.loc[(dfROC.prob > thresholds[i]) & (dfROC.type=='TP')])
    fpc += len(dfROC.loc[(dfROC.prob > thresholds[i]) & (dfROC.type!='TP')])
              
    # print('TP amount {} in threshold {}'.format(tpc, thresholds[i]))
    # print('FP amount {} in threshold {}'.format(fpc, thresholds[i]))
    

    tpr.append( tpc/(tp+fn) )
    fppi.append( fpc/normals )
    tpc = 0
    fpc = 0
  
  gamma = [i for i in fppi if i <= 50 ]
  print(max(fppi))
  auc_total = trapz(tpr, x=fppi)/max(fppi)
  print(max(gamma))
  auc_final = trapz(tpr[0:len(gamma)], x=gamma)/max(gamma)

  print('AUC TOTAL:', auc_total)
  fig, ax = plt.subplots()
  ax.plot(fppi, tpr)
  ax.set_xlabel('fPpI', fontsize=15)
  ax.set_ylabel('TPR', fontsize=15)
  ax.grid(True)
  plt.ylim(0,1)
  plt.show()


  print('AUC final:', auc_final)
  fig1, ax1 = plt.subplots()
  ax1.plot(gamma, tpr[0:len(gamma)])
  ax1.set_xlabel('fPpI', fontsize=12)
  ax1.set_ylabel('TPR', fontsize=12)
  ax1.grid(True)
  plt.ylim(0,1)
  plt.xlim(0,50)

  plt.savefig(os.path.join(MODELS_DIR,
                          'negative_pool_scaled', 
                           name+'.eps'), format='eps')
  
  plt.show()
  
  final_f1 = tp / (tp + (fp + fn)/2)

  row_results = [clf_name, auc_total, auc_final, final_f1]
  results_df = results_df.append(pd.DataFrame([row_results], columns=["model", "AUC_TOTAL", "AUC_final_50fPpi", "F1_final"]))
  results_df.to_csv(os.path.join(experiments_results_path, results_filename), index=False)

  confmat_results = [clf_name, tp, fp, fn, tp+fn]
  results_confmat = results_confmat.append(pd.DataFrame([confmat_results], columns=["model", "TP", "FP", "FN", "Total True"]))
  results_confmat.to_csv(os.path.join(experiments_results_path, confusion_matrix_filename), index=False)


In [66]:
def load_and_predict(selectedFold, classifier_name):

  #Reading data
  dfs_train, dfs_test = train_test_function()
  print('Length of train set: ', len(dfs_train))
  print('Length of test set: ', len(dfs_test))

  dfs_tr_hyperopt = dfs_train.sample(frac=0.1, random_state=42)
  dfs_ts_hyperopt = dfs_test.sample(frac=0.1, random_state=42)

  dfs_train = dfs_train.append(dfs_tr_hyperopt).drop_duplicates(keep=False)
  dfs_test = dfs_test.append(dfs_ts_hyperopt).drop_duplicates(keep=False)
  print('Length of train set: ', len(dfs_train))
  print('Length of test set: ', len(dfs_test))

  try:  
    if dfs_train.isnull().values.any():
      colm = dfs_train.columns[dfs_train.isna().any()]
      dfs_train = dfs_train[dfs_train[colm[0]].notna()]

    if dfs_test.isnull().values.any():
      colm = dfs_test.columns[dfs_test.isna().any()]    
      dfs_test = dfs_test[dfs_test[colm[0]].notna()]

    print("Train dfs without nans: ", len(dfs_train))
    print("Test dfs without nans: ",len(dfs_test))

  except Exception as e:
    print("No nan values to drop, or not columns")
    print(e)

  y_train = dfs_train['label'].reset_index(drop=True)
  candidates_train = dfs_train[['name', 'label', 'x', 'y']].reset_index(drop=True)
  X_train = dfs_train.drop(['name', 'label',  'x', 'y'], axis=1).reset_index(drop=True)

  y_test = dfs_test['label'].reset_index(drop=True)
  candidates_test = dfs_test[['name', 'label', 'x', 'y']].reset_index(drop=True)
  X_test = dfs_test.drop(['name', 'label',  'x', 'y'], axis=1).reset_index(drop=True)


  #Obtain train-test datasets
  if selectedFold == 2:
    chosenCandidates = candidates_train
    trainDataFull = pd.DataFrame(X_test).copy()
    ss_f2 = StandardScaler()
    trainDataFull = pd.DataFrame(ss_f2.fit_transform(trainDataFull), columns=X_test.columns)
    trainDataFull['label'] = y_test

    model = pickle.load(open(os.path.join(MODELS_DIR,'negative_pool_scaled', f'best_model_{classifier_name}_{selectedFold}_{pipeFolder}_negative_pool_scaled.pkl'), 'rb'))
    y_pred = model.predict(pd.DataFrame(ss_f2.transform(X_train.copy()), columns=X_train.columns))
    y_pred_proba = model.predict_proba(pd.DataFrame(ss_f2.transform(X_train.copy()), columns=X_train.columns))[:,1]

  else:
    chosenCandidates = candidates_test
    trainDataFull = pd.DataFrame(X_train).copy()
    ss_f1 = StandardScaler()
    trainDataFull = pd.DataFrame(ss_f1.fit_transform(trainDataFull), columns=X_train.columns)
    trainDataFull['label'] = y_train

    model = pickle.load(open(os.path.join(MODELS_DIR,'negative_pool_scaled', f'best_model_{classifier_name}_{selectedFold}_{pipeFolder}_negative_pool_scaled.pkl'), 'rb'))
    y_pred = model.predict(pd.DataFrame(ss_f1.transform(X_test.copy()), columns=X_test.columns))
    y_pred_proba = model.predict_proba(pd.DataFrame(ss_f1.transform(X_test.copy()), columns=X_test.columns))[:,1]


  normals = []
  with open(os.path.join(RESULTS_DIR_MAIN,'normals_final.txt')) as f:
      for line in f:
          normals.append(line[:-1])

  if 'glcm' in pipeFolder:
    imageKeyName = 2
  else:
    imageKeyName = 3

  candidates_unique = [cand.split('_')[imageKeyName] for cand in chosenCandidates['name'].values.tolist()]
  candidates_unique = list(set(candidates_unique))

  groundTruthShort = [i for i in groundTruths if i.split('_')[0] in candidates_unique]
  print(len(groundTruthShort))

  normals = [i for i in normals if i in candidates_unique]

  print("Calculating fROC file...") 
  fn, dfROC, filename_key = calculateFROC(groundTruthShort, normals, chosenCandidates, y_pred_proba, classifier_name, pipeFolder+'_loaded_')
  print(len(dfROC.prob.unique()))

  return fn, dfROC, filename_key


In [67]:
def generate_fold_results(selected_candidates, normals, imageKeyLocation, trained_clf, y_test, y_pred, y_pred_proba, selectedFold, classifier):

  df_cm = confusion_matrix(y_test, y_pred )
  plt.figure(figsize=(5,5))
  sns.heatmap(df_cm, annot=True, fmt='d') # font size
  plt.show()

  filename = f'best_model_{classifier}_{selectedFold}_{pipeFolder}_negative_pool_scaled.pkl'
  pickle.dump(trained_clf, open(os.path.join(MODELS_DIR,'negative_pool_scaled', filename), 'wb'))
  print(f"Fold {selectedFold} of {classifier} was saved!")  

  candidates_unique = [cand.split('_')[imageKeyLocation] for cand in selected_candidates['name'].values.tolist()]

  candidates_unique = list(set(candidates_unique))
  groundTruthShort = [i for i in groundTruths if i.split('_')[0] in candidates_unique]

  print(len(groundTruthShort))

  normals = [i for i in normals if i in candidates_unique]

  print("Calculating fROC file...")
  fn, dfROC, filename_key = calculateFROC(groundTruthShort, normals, selected_candidates, y_pred_proba, classifier, pipeFolder)

  return fn, dfROC, filename_key

# **Main**

In [ ]:
# def main():
from sklearn.utils import shuffle


normals = []
with open(os.path.join(RESULTS_DIR_MAIN,'normals_final.txt')) as f:
  for line in f:
      normals.append(line[:-1])

if 'glcm' in pipeFolder:
  imageKeyName = 2
  usePCA = False
else:
  imageKeyName = 3
  usePCA = True


calibration_classifiers = ['adaboost', 'svm']

classifiers = [ 'gb', 'rf', 'logreg', 'knn', 'dt']
best_hyperparameters = dict()
y_pred_proba = []
f1_scores = []

# Creating the results file
experiments_results_path = os.path.join(MODELS_DIR, 'negative_pool_scaled')
results_filename = f'results_negative_pool_scaled_{pipeFolder}.csv'

if results_filename not in os.listdir(experiments_results_path):
  results_df = pd.DataFrame(columns=["model", "pAUC_mean", "f1_mean", "mcc_mean", "pAUC", "f1", "mcc", "selectedFold", "parameters"])
  results_df.to_csv(os.path.join(experiments_results_path, results_filename), index=False)
else:
  results_df = pd.read_csv(os.path.join(experiments_results_path, results_filename))


#Reading data
dfs_train, dfs_test = train_test_function()
print('Length of train set: ', len(dfs_train))
print('Length of test set: ', len(dfs_test))

dfs_tr_hyperopt = dfs_train.sample(frac=0.1, random_state=42)
dfs_ts_hyperopt = dfs_test.sample(frac=0.1, random_state=42)

dfs_train = dfs_train.append(dfs_tr_hyperopt).drop_duplicates(keep=False)
dfs_test = dfs_test.append(dfs_ts_hyperopt).drop_duplicates(keep=False)
print('Length of train set: ', len(dfs_train))
print('Length of test set: ', len(dfs_test))

dfs_hyperOpt = shuffle(dfs_tr_hyperopt.append(dfs_ts_hyperopt))


#Preprocessing steps
#Manage unbalanced data
# dataBalanced, dfNegative = rus(dfs) #Random majority undersampling

try:  
  if dfs_train.isnull().values.any():
    colm = dfs_train.columns[dfs_train.isna().any()]
    for c in colm:
      dfs_train = dfs_train[dfs_train[c].notna()]

  if dfs_test.isnull().values.any():
    colm = dfs_test.columns[dfs_test.isna().any()]    
    for c in colm:
      dfs_test = dfs_test[dfs_test[c].notna()]

  if dfs_hyperOpt.isnull().values.any():
    colm = dfs_hyperOpt.columns[dfs_hyperOpt.isna().any()]
    for c in colm:
      dfs_hyperOpt = dfs_hyperOpt[dfs_hyperOpt[c].notna()]

  print("Train dfs without nans: ", len(dfs_train))
  print("Test dfs without nans: ",len(dfs_test))

except Exception as e:
  print("No nan values to drop, or not columns")
  print(e)

# DROP THE CORRELATED VARIABLES
cor_matrix = dfs_hyperOpt.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print(to_drop)

dfs_hyperOpt = dfs_hyperOpt.drop([i for i in dfs_hyperOpt.columns if i in to_drop], axis=1)
dfs_train = dfs_train.drop([i for i in dfs_train.columns if i in to_drop], axis=1)
dfs_test = dfs_test.drop([i for i in dfs_test.columns if i in to_drop], axis=1)

# Data Splitting

y_train = dfs_train['label'].reset_index(drop=True)
candidates_train = dfs_train[['name', 'label', 'x', 'y']].reset_index(drop=True)
X_train = dfs_train.drop(['name', 'label',  'x', 'y'], axis=1).reset_index(drop=True)

y_test = dfs_test['label'].reset_index(drop=True)
candidates_test = dfs_test[['name', 'label', 'x', 'y']].reset_index(drop=True)
X_test = dfs_test.drop(['name', 'label',  'x', 'y'], axis=1).reset_index(drop=True)

# Normalization and balancing of data

# Data for FOLD 1
trainDataFull = pd.DataFrame(X_train).copy()
ss_f1 = StandardScaler()
trainDataFull = pd.DataFrame(ss_f1.fit_transform(trainDataFull), columns=X_train.columns)
if usePCA:
  # Make an instance of the Model
  pca = PCA(.95)
  trainDataFull = pd.DataFrame(pca.fit_transform(trainDataFull))

trainDataFull['label'] = y_train
dataBalanced, dfNegative = rus(trainDataFull)
print("Data Balanced fold 1: ", len(dataBalanced))
print("Negative pool fold 1: ",len(dfNegative))


# Data for FOLD 2
trainDataFull_2 = pd.DataFrame(X_test).copy()
ss_f2 = StandardScaler()
trainDataFull_2 = pd.DataFrame(ss_f2.fit_transform(trainDataFull_2), columns=X_test.columns)
if usePCA:
  # Make an instance of the Model
  pca2 = PCA(.95)
  trainDataFull_2 = pd.DataFrame(pca2.fit_transform(trainDataFull_2))    

trainDataFull_2['label'] = y_test

dataBalanced_2, dfNegative_2 = rus(trainDataFull_2)
print("Data Balanced fold 2: ", len(dataBalanced_2))
print("Negative pool fold 2: ",len(dfNegative_2))



hyperOptBalanceData = dfs_hyperOpt.drop(['name',  'x', 'y'], axis=1)
ss = StandardScaler()
hyperOptBalanceDataScaled = pd.DataFrame(ss.fit_transform(hyperOptBalanceData.drop('label', axis=1)), columns=X_train.columns)
if usePCA:
  # Make an instance of the Model
  pca_hp = PCA(.95)
  hyperOptBalanceDataScaled = pd.DataFrame(pca_hp.fit_transform(hyperOptBalanceDataScaled))

hyperOptBalanceDataScaled['label'] = hyperOptBalanceData['label'].values

cvf1Score = []
cvMccScore = []
cvpAUCScore = []


for classifier in classifiers:

    print("Training new model...")
    print('******************** RESULTS FOLD 1 **********************')

    prediction_set_1 = pd.DataFrame(ss_f1.transform(X_test.copy()), columns=X_test.columns)
    if usePCA:
      prediction_set_1 = pd.DataFrame(pca.transform(prediction_set_1))


    clf, best_params, y_pred_proba, y_pred, f1_score_1 = negative_pool(dataBalanced.copy(), dfNegative.copy(), hyperOptBalanceDataScaled, 
                                                                      prediction_set_1, y_test, classifier, classifier in calibration_classifiers)
    best_hyperparameters[classifier] = best_params

    cvf1Score.append(f1_score(y_test, y_pred, average='binary'))
    cvMccScore.append(matthews_corrcoef(y_test, y_pred))
    cvpAUCScore.append(roc_auc_score(y_test, y_pred_proba, max_fpr = 0.0001))

    fn_1, dfROC_1, filename_key_1 = generate_fold_results(candidates_test, normals, imageKeyName, clf, y_test, y_pred, y_pred_proba, 1, classifier)
    
    print('******************** RESULTS FOLD 2 **********************')
 
    prediction_set_2 = pd.DataFrame(ss_f2.transform(X_train.copy()), columns=X_train.columns)
    if usePCA:
      prediction_set_2 = pd.DataFrame(pca2.transform(prediction_set_2))

    clf2, best_params, y_pred_proba_2, y_pred_2, f1_score_2 = negative_pool(dataBalanced_2.copy(), dfNegative_2.copy(), hyperOptBalanceDataScaled, 
                                                                            prediction_set_2, y_train, classifier, classifier in calibration_classifiers, best_params)  
    cvf1Score.append(f1_score(y_train, y_pred_2, average='binary'))
    cvMccScore.append(matthews_corrcoef(y_train, y_pred_2))
    cvpAUCScore.append(roc_auc_score(y_train, y_pred_proba_2, max_fpr = 0.0001))

    fn_2, dfROC_2, filename_key_2 = generate_fold_results(candidates_train, normals, imageKeyName, clf2, y_train, y_pred_2, y_pred_proba_2, 2, classifier)


    if np.array(cvf1Score).argmax() == 0:
      best_f1 = cvf1Score[0]
      best_mcc = cvMccScore[0]
      best_pAUC = cvpAUCScore[0]

    else:
      best_f1 = cvf1Score[1]
      best_mcc = cvMccScore[1]
      best_pAUC = cvpAUCScore[1]

    print("cvf1Score: ", cvf1Score)

    row_results = [classifier, np.mean(cvpAUCScore), np.mean(cvf1Score), np.mean(cvMccScore), best_pAUC, best_f1, best_mcc, np.array(cvf1Score).argmax()+1, best_params]
    results_df = results_df.append(pd.DataFrame([row_results], columns=["model", "pAUC_mean", "f1_mean", "mcc_mean", "pAUC", "f1", "mcc", "selectedFold", "parameters"]))
    results_df.to_csv(os.path.join(experiments_results_path, results_filename), index=False)

    dfROC_1 = dfROC_1.append(dfROC_2) 

    print(len(dfROC_1.prob.unique()))
    draw_curve(fn_1+fn_2, len(normals), dfROC_1, f'FROC_curve_{classifier}_pip{pipeFolder}_fn{fn_1+fn_2}_normals{len(normals)}_negative_pool_scaled', classifier)

    del clf, y_pred_proba, y_pred, f1_score_1, clf2, y_pred_proba_2, y_pred_2, f1_score_2, best_f1, best_mcc, best_pAUC, dfROC_1, dfROC_2


Length of train set:  378938
Length of test set:  393246
Length of train set:  341044
Length of test set:  353921
Train dfs without nans:  337133
Test dfs without nans:  350004


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


['Median_LL1', 'sumVariance_LL1', 'sumAverage_LL1', 'sumAverage_LL2', 'Entropy_LH2', 'sumAverage_LH2', 'Variance_HL2', 'kurtosis_HL2', 'diffEntropy_HL2', 'sumAverage_HL2', 'homogeneity_LH1', 'sumEntropy_LH1', 'diffEntropy_LH1', 'sumAverage_LH1', 'sumEntropy_HL1', 'diffEntropy_HL1', 'sumAverage_HL1', 'skewness_HH1', 'kurtosis_HH1', 'Entropy_HH1', 'CoefficientOfVariation_HH1', 'homogeneity_HH1', 'sumEntropy_HH1', 'diffEntropy_HH1', 'sumAverage_HH1']
rus negative 1:  330767
rus negative 1 without duplicates:  330742
df_0:  6366
append  337133
df_negative drop no keeping 330729
after drop duplicates with appended:  324363
Length of balanced data:  12732
0    6366
1    6366
Name: label, dtype: int64
Data Balanced fold 1:  12732
Negative pool fold 1:  324363
rus negative 1:  346674
rus negative 1 without duplicates:  346610
df_0:  3330
append  350004
df_negative drop no keeping 346583
after drop duplicates with appended:  343254
Length of balanced data:  6660
0    3330
1    3330
Name: label,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
# if __name__== "__main__":
#   main()